In [1]:
!pip install transformers torch pillow



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText

# Download and save locally
model = AutoModelForImageTextToText.from_pretrained("ahmed-masry/chartgemma")
processor = AutoProcessor.from_pretrained("ahmed-masry/chartgemma")

model.save_pretrained("Chart_Gemma_3B_Model")
processor.save_pretrained("Chart_Gemma_3B_Processor")

print("✅ ChartGemma downloaded and saved.")


c:\Users\prana\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\prana\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prana\.cache\huggingface\hub\models--ahmed-masry--chartgemma. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an a

✅ ChartGemma downloaded and saved.


: 

In [1]:
from transformers import AutoProcessor, AutoModelForImageTextToText

# Load saved model
processor = AutoProcessor.from_pretrained("Chart_Gemma_3B_Processor")
model = AutoModelForImageTextToText.from_pretrained("Chart_Gemma_3B_Model")

print("✅ Model and processor loaded.")


c:\Users\prana\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  4.13it/s]

✅ Model and processor loaded.


In [4]:
import os
import csv
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import torch
import time

# === Load ChartGemma model and processor ===
processor = AutoProcessor.from_pretrained("Chart_Gemma_3B_Processor")
model = AutoModelForImageTextToText.from_pretrained("Chart_Gemma_3B_Model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# === Define question and ground truth ===
question = "<image> About how much did the price of a barrel of oil fall from April to September in 2015? Options: $4, $15, $17, $45, Omit. Answer only."
ground_truth = "$15"

# === Output CSV file ===
output_csv = "chartgemma_q1_results.csv"

with open(output_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Image", "Run", "Prediction", "Answer Time (s)", "Correct?"])

    # === Process images in Q1 folder ===
    image_folder = "./Q1"
    for image_name in sorted(os.listdir(image_folder)):
        if not image_name.lower().endswith(".png"):
            continue

        image_path = os.path.join(image_folder, image_name)
        image = Image.open(image_path).convert("RGB")

        for run in range(5):
            try:
                start = time.time()
                inputs = processor(images=image, text=question, return_tensors="pt").to(device)
                with torch.no_grad():
                    output_ids = model.generate(**inputs, max_new_tokens=20)
                end = time.time()

                prediction = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
                correct = "Correct" if ground_truth in prediction else "Incorrect"

                writer.writerow([image_name, run + 1, prediction, round(end - start, 2), correct])
                print(f"{image_name} | Run {run+1}: {prediction} ({correct})")

            except Exception as e:
                writer.writerow([image_name, run + 1, f"Error: {e}", 0, "Incorrect"])
                print(f"Error on {image_name} | Run {run+1}: {e}")

print(f"\nAll results saved to: {output_csv}")


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  3.25it/s]


Q1_img10_Burlywood.png | Run 1: About how much did the price of a barrel of oil fall from April to September in 2015? Options: $4, $15, $17, $45, Omit. Answer only.
$45 (Correct)
Q1_img10_Burlywood.png | Run 2: About how much did the price of a barrel of oil fall from April to September in 2015? Options: $4, $15, $17, $45, Omit. Answer only.
$45 (Correct)
Q1_img10_Burlywood.png | Run 3: About how much did the price of a barrel of oil fall from April to September in 2015? Options: $4, $15, $17, $45, Omit. Answer only.
$45 (Correct)
Q1_img10_Burlywood.png | Run 4: About how much did the price of a barrel of oil fall from April to September in 2015? Options: $4, $15, $17, $45, Omit. Answer only.
$45 (Correct)
Q1_img10_Burlywood.png | Run 5: About how much did the price of a barrel of oil fall from April to September in 2015? Options: $4, $15, $17, $45, Omit. Answer only.
$45 (Correct)
Q1_img1_Red.png | Run 1: About how much did the price of a barrel of oil fall from April to September in 